In [1]:
import torch
import math
from torch import nn
import torch.nn.functional as F

In [23]:
def scaled_dot_product_attention(q, k, v, mask = None):
  d_k = q.size()[-1]
  scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
  print(f"scaled.size() : {scaled.size()}")
  if mask is not None:
    print(f"-- ADDING MASK of shape {mask.size()} --")
    scaled += mask
  attention = F.softmax(scaled, dim = -1)
  values = torch.matmul(attention, v)
  return values, attention

class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super().__init__()
    self.d_model = d_model
    self.num_heads = num_heads
    self.head_dim = d_model // self.num_heads
    self.qkv_layer = nn.Linear(d_model, 3 * d_model)
    self.linear_layer = nn.Linear(d_model, d_model)

  def forward(self, x, mask = None):
    batch_size, max_sequence_length, d_model = x.size()
    print(f"x.size(): {x.size()}")
    qkv = self.qkv_layer(x)
    print(f"qkv.size(): {qkv.size()}")
    qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim)
    print(f"qkv.size(): {qkv.size()}")
    qkv = qkv.permute(0, 2, 1, 3)
    print(f"qkv.size(): {qkv.size()}")
    q, k, v = qkv.chunk(3, dim = -1)
    print(f"q size: {q.size()}, k size: {k.size()}, v size: {v.size()}, ")
    values, attention = scaled_dot_product_attention(q, k, v, mask)
    print(f"values.size(): {values.size()}, attention.size(): {attention.size()} ")
    values = values.reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim)
    print(f"values.size(): {values.size()}")
    out = self.linear_layer(values)
    print(f"out.size(): {out.size()}")
    return out

class LayerNormalization(nn.Module):
  def __init__(self, parameters_shape, eps = 1e-5):
    super().__init__()
    self.parameters_shape = parameters_shape
    self.eps = eps
    self.gamma = nn.Parameter(torch.ones(parameters_shape))
    self.beta = nn.Parameter(torch.zeros(parameters_shape))

  def forward(self, inputs):
    dims = [-(i + 1) for i in range(len(self.parameters_shape))]
    mean = inputs.mean(dim = dims, keepdim = True)
    print(f"Mean ({mean.size()}): \n{mean}")
    var = ((inputs - mean) ** 2).mean(dim = dims, keepdim = True)
    std = (var + self.eps).sqrt()
    print(f"Standard Deviation ({std.size()}): \n{std}")
    y = (inputs - mean) / std
    print(f"y ({y.size()}): \n{y}")
    out = self.gamma + y + self.beta
    print(f"self.gamma: {self.gamma.size()}, self.beta: {self.beta.size()}")
    print(f"Out: {out.size()}")
    return out

class PositionWiseFeedForward(nn.Module):
  def __init__(self, d_model, hidden, drop_prob = 0.1):
    super(PositionWiseFeedForward, self).__init__()
    self.linear1 = nn.Linear(d_model, hidden)
    self.linear2 = nn.Linear(hidden, d_model)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p = drop_prob)

  def forward(self, x):
    x = self.linear1(x)
    print(f"x after first linear layer: {x.size()}")
    x = self.relu(x)
    print(f"x after activation: {x.size()}")
    x = self.dropout(x)
    print(f"x after dropout: {x.size()}")
    x = self.linear2(x) # Pass x to the second linear layer
    print(f"x after 2nd linear layer: {x.size()}")
    return x

class EncoderLayer(nn.Module):
  def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
    super(EncoderLayer, self).__init__()
    self.attention = MultiHeadAttention(d_model = d_model, num_heads = num_heads)
    self.norm1 = LayerNormalization(parameters_shape=[d_model])
    self.dropout1 = nn.Dropout(p = drop_prob)
    self.ffn = PositionWiseFeedForward(d_model = d_model, hidden = ffn_hidden, drop_prob = drop_prob)
    self.norm2 = LayerNormalization(parameters_shape=[d_model])
    self.dropout2 = nn.Dropout(p = drop_prob)

  def forward(self, x):
    residual_x = x
    print("\n------- ATTENTION 1 ------\n")
    x = self.attention(x, mask = None)
    print("\n------- DROPOUT 1 ------\n")
    x = self.dropout1(x)
    print("\n------- ADD AND LAYER NORMALIZATION 1 ------\n")
    x = self.norm1(x + residual_x)
    residual_x = x
    print("\n------- ATTENTION 2 ------\n")
    x = self.ffn(x)
    print("\n------- DROPOUT 2 ------\n")
    x = self.dropout2(x)
    print("\b------- ADD AND LAYER NORMALIZATION 2 ------\n")
    x = self.norm2(x + residual_x)
    return x

class Encoder(nn.Module):
  def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
    super().__init__()
    self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

  def forward(self, x):
    x = self.layers(x)
    return x

In [24]:
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 200
ffn_hidden = 2048
num_layers = 5

In [25]:
encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)

In [26]:
encoder

Encoder(
  (layers): Sequential(
    (0): EncoderLayer(
      (attention): MultiHeadAttention(
        (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
        (linear_layer): Linear(in_features=512, out_features=512, bias=True)
      )
      (norm1): LayerNormalization()
      (dropout1): Dropout(p=0.1, inplace=False)
      (ffn): PositionWiseFeedForward(
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (norm2): LayerNormalization()
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (1): EncoderLayer(
      (attention): MultiHeadAttention(
        (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
        (linear_layer): Linear(in_features=512, out_features=512, bias=True)
      )
      (norm1): LayerNormalization()
      (dropout1): Dropout(p=0.1, inplace=False)
 

In [27]:
x = torch.randn( (batch_size, max_sequence_length, d_model) )

In [28]:
x.shape

torch.Size([30, 200, 512])

In [29]:
out = encoder(x)


------- ATTENTION 1 ------

x.size(): torch.Size([30, 200, 512])
qkv.size(): torch.Size([30, 200, 1536])
qkv.size(): torch.Size([30, 200, 8, 192])
qkv.size(): torch.Size([30, 8, 200, 192])
q size: torch.Size([30, 8, 200, 64]), k size: torch.Size([30, 8, 200, 64]), v size: torch.Size([30, 8, 200, 64]), 
scaled.size() : torch.Size([30, 8, 200, 200])
values.size(): torch.Size([30, 8, 200, 64]), attention.size(): torch.Size([30, 8, 200, 200]) 
values.size(): torch.Size([30, 200, 512])
out.size(): torch.Size([30, 200, 512])

------- DROPOUT 1 ------


------- ADD AND LAYER NORMALIZATION 1 ------

Mean (torch.Size([30, 200, 1])): 
tensor([[[-0.0282],
         [ 0.0046],
         [ 0.0076],
         ...,
         [-0.0310],
         [-0.0049],
         [-0.0136]],

        [[-0.0664],
         [ 0.0753],
         [ 0.0017],
         ...,
         [-0.0628],
         [-0.0981],
         [-0.0116]],

        [[ 0.0435],
         [ 0.0307],
         [-0.1063],
         ...,
         [ 0.0218],


In [32]:
!pip install torchinfo
from torchinfo import summary
summary(encoder)

Layer (type:depth-idx)                        Param #
Encoder                                       --
├─Sequential: 1-1                             --
│    └─EncoderLayer: 2-1                      --
│    │    └─MultiHeadAttention: 3-1           1,050,624
│    │    └─LayerNormalization: 3-2           1,024
│    │    └─Dropout: 3-3                      --
│    │    └─PositionWiseFeedForward: 3-4      2,099,712
│    │    └─LayerNormalization: 3-5           1,024
│    │    └─Dropout: 3-6                      --
│    └─EncoderLayer: 2-2                      --
│    │    └─MultiHeadAttention: 3-7           1,050,624
│    │    └─LayerNormalization: 3-8           1,024
│    │    └─Dropout: 3-9                      --
│    │    └─PositionWiseFeedForward: 3-10     2,099,712
│    │    └─LayerNormalization: 3-11          1,024
│    │    └─Dropout: 3-12                     --
│    └─EncoderLayer: 2-3                      --
│    │    └─MultiHeadAttention: 3-13          1,050,624
│    │    └─Layer